In [6]:
#We need these in this file:
import load_read_name_extractor as lrne
import SVM_classifier_general as SVM_GENERAL
import os
import pickle

In [8]:
features_load = np.load('features/hog_features_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
names_load = np.load('features/img_names_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

4935
4935


In [9]:
labels = get_lodging_scores.get_labels(5)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
ht_mean = features_load[0].mean(axis=0)
ht_mean

print(labels[0])

[0.20888097 0.16330611 0.04696022 ... 0.06301632 0.05749686 0.03150874]
(61, 50, '160607_Min', 2.0)


In [10]:
my_set = set(map(tuple, labels))
# convert set back to list of arrays
labels = list(map(np.array, my_set))

In [12]:
names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

In [15]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame, error = svm_general.match_pic_label(features_load, labels, names)
#data_frame_names, error_names = match_pic_label_names(features_load, labels, names)
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


KeyboardInterrupt: 

In [ ]:
percentage = 80
partition = int(len(features_aar)*percentage/100)
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)
clf.fit(x_train,y_train)

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
filename = 'SVM_linear_64_128_3bins_cells_8x8_block_2x2_balanced.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.7197518097207859


              precision    recall  f1-score   support

         1.0       0.85      0.83      0.84       684
         2.0       0.38      0.46      0.41       193
         3.0       0.59      0.46      0.52        90

    accuracy                           0.72       967
   macro avg       0.61      0.58      0.59       967
weighted avg       0.73      0.72      0.73       967



In [ ]:
# Define cost matrix
num_classes = 20
cost_matrix = [[0]*num_classes for i in range(num_classes)]

for i in range(num_classes):
    for j in range(num_classes):
        cost_matrix[i][j] = abs(i-j)
cost_matrix

In [ ]:
def custom_scorer(y_true, y_pred):
    """
    Calculates the cost of classifying class i as class j.
    :param y_true: true labels
    :param y_pred: predicted labels
    :return: total cost
    """
    cost = 0
    for i in range(len(y_true)):
        cost += cost_matrix[int(y_true[i])][int(y_pred[i])]
    return cost

# Create scorer
scorer = make_scorer(custom_scorer, greater_is_better=False)

In [ ]:
# Define SVM model
svm = clf

# Define parameter grid for grid search
param_grid = {'C': np.logspace(-3, 3, 6)}

# Perform grid search using custom scorer
grid_search = GridSearchCV(svm, param_grid, scoring=scorer)
grid_search.fit(x_train,y_train)

# Print best parameters and score
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
y_pred = grid_search.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))